In [6]:
import cv2
import numpy as np
import pickle

def encoder(video_path, color_transform, downsampling_ratio):
    # Load video file
    cap = cv2.VideoCapture(video_path)

    # Create a list to store encoded frames
    encoded_frames = []

    # Read and process video frames
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Apply color transform
        frame_transformed = cv2.cvtColor(frame, color_transform)

        # Split channels
        y, cr, cb = cv2.split(frame_transformed)

        # Perform downsampling on chroma channels (Cr and Cb)
        y_downsampled = cv2.resize(y, None, fx=downsampling_ratio, fy=downsampling_ratio)
        cr_downsampled = cv2.resize(cr, None, fx=downsampling_ratio, fy=downsampling_ratio)
        cb_downsampled = cv2.resize(cb, None, fx=downsampling_ratio, fy=downsampling_ratio)

        #print(y_downsampled.dtype)

        # Convert frame to int8 or uint8 representation
        y_downsampled_encoded = y_downsampled.astype(np.uint8)  # Keep the luminance channel as is
        cr_downsampled_encoded = cr_downsampled.astype(np.uint8)  # or np.uint8
        cb_downsampled_encoded = cb_downsampled.astype(np.uint8)  # or np.uint8

        # merge back
        frame_encoded = cv2.merge((y_downsampled_encoded, cr_downsampled_encoded, cb_downsampled_encoded))
        #frame_encoded = cv2.merge((y_downsampled, cr_downsampled, cb_downsampled))


        # Store the encoded frame components
        #encoded_frames.append((y_encoded, cr_downsampled_encoded, cb_downsampled_encoded))
        encoded_frames.append(frame_encoded)

    cap.release()

    # Store the encoded frames in a pickle file
    encoded_file_path = 'encoded.pickle'
    with open(encoded_file_path, 'wb') as f:
        pickle.dump(encoded_frames, f)

    return encoded_file_path




In [7]:
def decoder(encoded_file_path):
    # Read the saved encoded video from the pickle file
    with open(encoded_file_path, 'rb') as f:
        encoded_frames = pickle.load(f)

    # Get frame dimensions
    #height, width = encoded_frames[0][0].shape
    height , width = (480, 640)
    print(height,width)

    # Define the codec and create a VideoWriter object (in this case, it is MP4)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Open an empty file to save the decoded video output
    out = cv2.VideoWriter('decoded.mp4', fourcc, 20.0, (width, height))

    # Iterate over encoded frames and decode them
    for frame_encoded in encoded_frames:
        # Separate the channels
        y_downsampled, cr_downsampled, cb_downsampled = cv2.split(frame_encoded)

        # Upsample the chroma channels
        y_upsampled = cv2.resize(y_downsampled, (width, height))
        cr_upsampled = cv2.resize(cr_downsampled, (width, height))
        cb_upsampled = cv2.resize(cb_downsampled, (width, height))

        # Merge the channels back
        frame_decoded = cv2.merge([y_upsampled, cr_upsampled, cb_upsampled])

        # Convert the frame to BGR color space
        frame_bgr = cv2.cvtColor(frame_decoded, cv2.COLOR_YCrCb2BGR)

        # Write the decoded frame to the output video file
        out.write(frame_bgr)

    out.release()
    return

In [11]:
# Encode the video
encoded_file_path = encoder('videorec.mp4', cv2.COLOR_BGR2YCrCb, 0.5)
print(encoded_file_path)

# Decode the video
decoder(encoded_file_path)


encoded.pickle
480 640


In [9]:
# Let us run your encoder and decoder
import os
print("Wait ... ")
encoder('videorec.mp4', cv2.COLOR_BGR2YCrCb, 0.5)
decoder(encoded_file_path)
print("Below are the statistics of your resultant decoded video")
print("")
compsize = os.path.getsize('encoded.pickle')
decsize = os.path.getsize('decoded.mp4')
logcompression = np.log(decsize/compsize)
print(compsize,decsize)
print("Log Compression ratio  =  ", logcompression)

Wait ... 
480 640
Below are the statistics of your resultant decoded video

12674706 212049
Log Compression ratio  =   -4.090546252883517


In [10]:
from moviepy.editor import *
clip = VideoFileClip("decoded.mp4")
clip.subclip(0,3)
clip.ipython_display()

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
